# Vigade analüüs

See märkmik analüüsib RAG-põhise kursuste soovitaja tagasiside logi ning arvutab vigade statistika vahesammude kaupa.

In [1]:
import pandas as pd
import numpy as np

## 1. Andmete laadimine

In [2]:
# Lae tagasiside logi
log_df = pd.read_csv("tagasiside_log.csv")
print(f"Kokku {len(log_df)} tagasiside kirjet")
log_df.head(10)

Kokku 2 tagasiside kirjet


,Aeg,Kasutaja päring,Filtrid,Leitud aine_koodid,Leitud nimed,LLM Vastus,Hinnang,Veatüüp,Kommentaar
0,2026-03-02 10:51:09,tahan origamiga tegeleda,"semester=kevad, sügis, eap=1.0-36.0, hindamiss...","['LTMS.00.086', 'LOOM.01.126', 'LTOM.04.010', ...",['Origami ja matemaatika: voltimise kunst ja t...,- **LTMS.00.086 – Origami ja matemaatika: volt...,Hea,NaN,NaN
1,2026-03-02 11:06:14,tahan 3d printimisega tegeleda,"semester=kevad, sügis, eap=1.0-36.0, hindamiss...","['LOTI.05.097', 'LOTI.05.039', 'MTAT.03.015', ...","['Tehniline joonestamine', 'Elektroonika ja pr...",- **LOTI.05.097 – Tehniline joonestamine** (3....,Hea,NaN,"Hea, aga Tehnilise joonestamise ainest oleks v..."


## 2. Testjuhtumite laadimine

In [3]:
test_df = pd.read_csv("testjuhtumid.csv")
test_df

,päring,oodatud_aine_koodid,oodatud_nimed
0,tahan tegeleda origamiga,LTMS.00.086,Origami ja matemaatika: voltimise kunst ja teadus
1,tahan 3d printimisega tegeleda,"LOTI.05.039, LOTI.05.097","Elektroonika ja prototüüpimise erilabor, Tehni..."
2,tahan õppida 3d disaini,"LOFY.01.021, LOTI.05.097","Insenerigraafika, Tehniline joonestamine"
3,Tahan õppida termodünaamika seaduseid,"LOKT.02.037, LOFY.04.004","Füüsikaline keemia 1. osa, Termodünaamika ja s..."
4,tahan õppida vaakumi kohta,LOFY.01.026,Vaakum- ja krüotehnika


## 3. Kõik tulemused

In [4]:
# Ülevaade kõikidest päringutest
overview = log_df[["Kasutaja päring", "Leitud aine_koodid", "Leitud nimed", "Hinnang", "Veatüüp"]].copy()
overview

,Kasutaja päring,Leitud aine_koodid,Leitud nimed,Hinnang,Veatüüp
0,tahan origamiga tegeleda,"['LTMS.00.086', 'LOOM.01.126', 'LTOM.04.010', ...",['Origami ja matemaatika: voltimise kunst ja t...,Hea,NaN
1,tahan 3d printimisega tegeleda,"['LOTI.05.097', 'LOTI.05.039', 'MTAT.03.015', ...","['Tehniline joonestamine', 'Elektroonika ja pr...",Hea,NaN


## 4. Halvad vs head tulemused

In [5]:
total = len(log_df)
good = (log_df["Hinnang"] == "Hea").sum()
bad = (log_df["Hinnang"] == "Halb").sum()

print(f"Kokku päringuid: {total}")
print(f"Head tulemused:  {good} ({good/total*100:.1f}%)")
print(f"Halvad tulemused: {bad} ({bad/total*100:.1f}%)")

Kokku päringuid: 2
Head tulemused:  2 (100.0%)
Halvad tulemused: 0 (0.0%)


## 5. Vigade analüüsi tabel vahesammude kaupa

Vahesammud:
1. **Metaandmete filtreerimine** — filtrid olid liiga karmid/valed
2. **RAGi vektorotsing** — otsing leidis valed ained
3. **LLMi vastuse genereerimine** — LLM hallutsineeris/vastas valesti

In [6]:
# Filtreeri ainult halvad tulemused
bad_df = log_df[log_df["Hinnang"] == "Halb"].copy()
bad_total = len(bad_df)

# Loenda vigade tüübid
error_map = {
    "Filtrid olid liiga karmid/valed": "Metaandmete filtreerimine",
    "RAG otsing leidis valed ained": "RAGi vektorotsing",
    "LLM hallutsineeris/vastas valesti": "LLMi vastuse genereerimine",
}

bad_df["Vahesamm"] = bad_df["Veatüüp"].map(error_map).fillna("Määramata")

error_counts = bad_df["Vahesamm"].value_counts()

# Koosta analüüsi tabel
steps = ["Metaandmete filtreerimine", "RAGi vektorotsing", "LLMi vastuse genereerimine"]
analysis = pd.DataFrame({
    "Vahesamm": steps,
    "Vigade arv": [error_counts.get(s, 0) for s in steps],
})
analysis["% kõikidest vigadest"] = (
    (analysis["Vigade arv"] / bad_total * 100).round(1) if bad_total > 0 else 0
)
analysis["% kõikidest päringutest"] = (
    (analysis["Vigade arv"] / total * 100).round(1) if total > 0 else 0
)

print(f"Halvad tulemused kokku: {bad_total}")
print()
analysis

Halvad tulemused kokku: 0



,Vahesamm,Vigade arv,% kõikidest vigadest,% kõikidest päringutest
0,Metaandmete filtreerimine,0,0,0.0
1,RAGi vektorotsing,0,0,0.0
2,LLMi vastuse genereerimine,0,0,0.0


## 6. Halvad juhtumid detailselt

In [7]:
if not bad_df.empty:
    display_cols = ["Kasutaja päring", "Leitud aine_koodid", "Leitud nimed", "Vahesamm"]
    bad_df[display_cols]
else:
    print("Halbu tulemusi ei ole!")

Halbu tulemusi ei ole!


## 7. Kokkuvõte

In [8]:
print("=" * 50)
print("VIGADE ANALÜÜSI KOKKUVÕTE")
print("=" * 50)
print(f"Päringuid kokku:        {total}")
print(f"Head tulemused:         {good} ({good/total*100:.1f}%)" if total > 0 else "Head tulemused: 0")
print(f"Halvad tulemused:       {bad} ({bad/total*100:.1f}%)" if total > 0 else "Halvad tulemused: 0")
print()
print("Vigade jaotus vahesammude kaupa:")
for _, row in analysis.iterrows():
    print(f"  {row['Vahesamm']}: {int(row['Vigade arv'])} ({row['% k\u00f5ikidest vigadest']}% vigadest, {row['% k\u00f5ikidest p\u00e4ringutest']}% k\u00f5ikidest)")

SyntaxError: f-string expression part cannot include a backslash (1271424036.py, line 10)